<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/cigarettes_model_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from PIL import Image

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
def load_model(model_path):
    model = models.detection.fasterrcnn_resnet50_fpn(weights=models.detection.FasterRCNN_ResNet50_FPN_Weights.COCO_V1)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    num_classes = 2
    model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()
    return model

In [11]:

# Funzione per eseguire il modello su un'immagine
def get_predictions(model, image, threshold=0.5):
    # Trasformazione dell'immagine in formato tensor
    transform = transforms.ToTensor()
    image_tensor = transform(image).unsqueeze(0)  # Aggiungi dimensione batch
    with torch.no_grad():
        predictions = model(image_tensor)

    # Estrai le bounding box, punteggi e etichette
    boxes = predictions[0]['boxes']
    scores = predictions[0]['scores']
    labels = predictions[0]['labels']

    # Filtra per score > threshold
    keep = scores > threshold
    boxes = boxes[keep]
    labels = labels[keep]
    return boxes, labels

In [12]:
# Funzione per disegnare le bounding box su un'immagine
def draw_boxes(image, boxes, labels):
    image = np.array(image)
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box.tolist()
        # Disegna la bounding box (etichetta è ignorata in questo esempio)
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    return image

In [13]:
# Funzione principale per eseguire tutto il processo
def process_images(model_path, images_folder, output_folder):
    model = load_model(model_path)

    # Crea la cartella di output se non esiste
    os.makedirs(output_folder, exist_ok=True)

    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        if image_path.endswith(('.jpg', '.png', '.jpeg')):
            print(f"Processing {image_name}")
            # Carica l'immagine
            image = Image.open(image_path).convert("RGB")
            boxes, labels = get_predictions(model, image)

            # Disegna le bounding box sull'immagine
            result_image = draw_boxes(image, boxes, labels)

            # Salva l'immagine con le bounding box
            result_image_path = os.path.join(output_folder, f"output_{image_name}")
            cv2.imwrite(result_image_path, result_image)
            print(f"Saved output to {result_image_path}")

In [14]:
def main():
    model_path = '/content/drive/MyDrive/pth_cigarette_detect/fasterrcnn_cigarette_final.pth'
    images_folder = '/content/drive/MyDrive/Photo/test'
    output_folder = '/content/drive/MyDrive/test_trained'

    process_images(model_path, images_folder, output_folder)

    #TODO risolvere tensor, box rossa e con la scritta, tqdm for output, best trained model

if __name__ == "__main__":
    main()

<ipython-input-10-2997e430d67c>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


Processing 20240928_140539_jpg.rf.a3f59ec5ebcc4dd94ed524a3659c45ec.jpg
Saved output to /content/drive/MyDrive/test_trained/output_20240928_140539_jpg.rf.a3f59ec5ebcc4dd94ed524a3659c45ec.jpg
Processing 20240928_133303_jpg.rf.6f615ecc8690a9068e521e60ebbc982b.jpg
Saved output to /content/drive/MyDrive/test_trained/output_20240928_133303_jpg.rf.6f615ecc8690a9068e521e60ebbc982b.jpg
Processing 20241120_110416_jpg.rf.479214375879a2a91444dbc7997d4af8.jpg
Saved output to /content/drive/MyDrive/test_trained/output_20241120_110416_jpg.rf.479214375879a2a91444dbc7997d4af8.jpg
Processing 20240928_122456_jpg.rf.499c83da73f30227543e134377dfc208.jpg
Saved output to /content/drive/MyDrive/test_trained/output_20240928_122456_jpg.rf.499c83da73f30227543e134377dfc208.jpg
Processing 20240928_142524_jpg.rf.8f95c2f0bd649fac886ea29e8c2e5932.jpg
Saved output to /content/drive/MyDrive/test_trained/output_20240928_142524_jpg.rf.8f95c2f0bd649fac886ea29e8c2e5932.jpg
Processing 20240928_133336_jpg.rf.5b2f60d5b6a6c3c1